In [14]:
# ==============================================================
# EURUSD – 4H Çift SMA Back-test (tqdm + adım bildirimli)
# ==============================================================

from pathlib import Path
import pandas as pd, numpy as np, vectorbt as vbt
from tqdm.auto import tqdm

# ---------- AYARLAR --------------------------------------------------------
DATA_DIR = Path(r"C:\DukascopyData")          # ← Parquet klasörü
FILE     = DATA_DIR / "EURUSD_1min.parquet"

FREQ_STR = "4h"                               # yıllıklaştırma frekansı
FAST_W   = np.arange(10, 41, 5)
SLOW_W   = np.arange(60, 181, 20)
MC_SIM   = 1000
# ---------------------------------------------------------------------------

print("► Veri okunuyor…")
price_1m = pd.read_parquet(FILE)["EURUSD"]
price_4h = price_1m.resample("4H").last().dropna()

split_date = price_4h.index[0] + pd.DateOffset(years=9)
price_in, price_out = price_4h.loc[:split_date], price_4h.loc[split_date:]

# -------- 1) PARAMETRE TARAMASI -------------------------------------------
print("\n=== 1) PARAMETRE TARAMASI BAŞLADI ===")
best_sharpe = -np.inf
best_fast = best_slow = None

grid_bar = tqdm(total=len(FAST_W) * len(SLOW_W), desc="Grid")
for f in FAST_W:
    for s in SLOW_W:
        f_ma = vbt.MA.run(price_in, window=f)
        s_ma = vbt.MA.run(price_in, window=s)
        entries = f_ma.ma_crossed_above(s_ma)
        exits   = f_ma.ma_crossed_below(s_ma)

        pf = vbt.Portfolio.from_signals(price_in, entries, exits)
        sharpe = pf.sharpe_ratio(freq=FREQ_STR)

        if sharpe > best_sharpe:
            best_sharpe, best_fast, best_slow = sharpe, f, s

        grid_bar.set_postfix({"fast": f, "slow": s, "Sharpe": f"{sharpe:.2f}"})
        grid_bar.update(1)

grid_bar.close()
print(f"→ En iyi Sharpe: {best_sharpe:.2f}  (fast={best_fast}, slow={best_slow})")

# -------- 2) OUT-OF-SAMPLE TEST -------------------------------------------

print("\n=== 2) OUT-OF-SAMPLE TEST BAŞLIYOR ===")
f_ma_o = vbt.MA.run(price_out, window=best_fast)
s_ma_o = vbt.MA.run(price_out, window=best_slow)

pf_o = vbt.Portfolio.from_signals(
    price_out,
    f_ma_o.ma_crossed_above(s_ma_o),
    f_ma_o.ma_crossed_below(s_ma_o)
)

oos_stats = pd.Series({
    "total_return":   pf_o.total_return(),
    "sharpe":         pf_o.sharpe_ratio(freq=FREQ_STR),
    "max_drawdown":   pf_o.max_drawdown()
})
print(oos_stats)
pf_o.plot().show()



# -------- 3) MONTE-CARLO ---------------------------------------------------
print("\n=== 3) MONTE-CARLO SİMÜLASYONU ===")
mc_bar = tqdm(range(MC_SIM), desc="Monte-Carlo")
mc_ret, mc_dd = [], []

for _ in mc_bar:
    mc_pf = pf_o.mc_simulate(n_simulations=1)
    mc_ret.append(mc_pf.mc_returns().iloc[0])
    mc_dd.append(mc_pf.mc_max_drawdowns().iloc[0])
    mc_bar.set_postfix({
        "ret%": f"{mc_ret[-1]*100:.1f}",
        "mdd%": f"{mc_dd[-1]*100:.1f}"
    })

mc_bar.close()
p5, p50, p95 = np.percentile(mc_ret, [5, 50, 95]) * 100
print(f"\nMonte-Carlo getirileri — 5/50/95 persentil: "
      f"{p5:.1f} %, {p50:.1f} %, {p95:.1f} %")


► Veri okunuyor…

=== 1) PARAMETRE TARAMASI BAŞLADI ===


C:\Users\PC\AppData\Local\Temp\ipykernel_2740\2095726279.py:21: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.



Grid:   0%|          | 0/49 [00:00<?, ?it/s]

→ En iyi Sharpe: -0.06  (fast=15, slow=60)

=== 2) OUT-OF-SAMPLE TEST BAŞLIYOR ===
total_return    0.084342
sharpe          1.279349
max_drawdown   -0.035873
dtype: float64


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed